### Import Divvy Bike Data: 

mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202201-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202202-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202203-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202204-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202205-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202206-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202207-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202208-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202209-divvy-publictripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202210-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202211-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline  202212-divvy-tripdata.csv<br>

### Import Weather Data: 
mongoimport --type csv -d chicago_bikes -c weather_daily --headerline weather_daily.csv

In [37]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

# Adding for query to find top ten stations 
#from pymongo.collection import Collection
#from pymongo.aggregation import Aggregation

In [15]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [16]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'travel_db', 'uk_food']


In [17]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [20]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_ridedata', 'weather_daily']


In [21]:
# review a document in the customer_list collection
print(db.divvy_ridedata.find_one())

{'_id': ObjectId('64e4eab4a682807a5695d165'), 'ride_id': 'C2F7DD78E82EC875', 'rideable_type': 'electric_bike', 'started_at': '2022-01-13 11:59:47', 'ended_at': '2022-01-13 12:02:44', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.0128005, 'start_lng': -87.665906, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual'}


In [22]:
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']

In [23]:
print(divvy_rides.count_documents({}))
print(weather_daily.count_documents({}))

5667717
365


In [24]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["started_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "started_at_date": date_part,
                    "started_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)


In [25]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["ended_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "ended_at_date": date_part,
                    "ended_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)

In [26]:
print(db.divvy_ridedata.find_one())
print(db.weather_daily.find_one())

{'_id': ObjectId('64e4eab4a682807a5695d165'), 'ride_id': 'C2F7DD78E82EC875', 'rideable_type': 'electric_bike', 'started_at': '2022-01-13 11:59:47', 'ended_at': '2022-01-13 12:02:44', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.0128005, 'start_lng': -87.665906, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-13', 'started_at_time': '11:59:47', 'ended_at_date': '2022-01-13', 'ended_at_time': '12:02:44'}
{'_id': ObjectId('64e4ebecb589d1838b565c4b'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01}


In [27]:
divvy_rides.create_index([("started_at_date", 1)])
weather_daily.create_index([("date", 1)])


'date_1'

In [28]:
pipeline = [
    {
        '$lookup': {
            'from': 'weather_daily',
            'localField': 'started_at_date',
            'foreignField': 'date',
            'as': 'weather_data'
        }
    },
    {
        '$unwind': {
            'path': '$weather_data',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$merge': {
            'into': 'divvy_ridedata_merged',  # Replace with your new collection name
            'whenMatched': 'merge',  # Merge documents with matching _id fields
            'whenNotMatched': 'insert'  # Insert documents that don't match
        }
    },
]

divvy_rides.aggregate(pipeline)
print("Update completed successfully.")


Update completed successfully.


In [70]:
divvy_ridedata_merged = db['divvy_ridedata_merged']
divvy_ridedata_merged.find_one()

{'_id': ObjectId('64e4163d948aee61d19bb3bc'),
 'afternoon_temp': 9.73,
 'cloud_cover': 17,
 'end_lat': 42.01256011541,
 'end_lng': -87.6743671152,
 'end_station_id': 'RP-007',
 'end_station_name': 'Clark St & Touhy Ave',
 'ended_at': '2022-01-10 08:46:17',
 'ended_at_date': '2022-01-10',
 'ended_at_time': '08:46:17',
 'evening_temp': 14,
 'max_temp': 20.84,
 'max_windspeed': 16.35,
 'member_casual': 'casual',
 'min_temp': 9.5,
 'morning_temp': 13.69,
 'night_temp': 20.75,
 'precipitation': 0.24,
 'ride_id': 'A6CF8980A652D272',
 'rideable_type': 'electric_bike',
 'start_lat': 42.012763,
 'start_lng': -87.6659675,
 'start_station_id': 525,
 'start_station_name': 'Glenwood Ave & Touhy Ave',
 'started_at': '2022-01-10 08:41:56',
 'started_at_date': '2022-01-10',
 'started_at_time': '08:41:56',
 'weather_data': {'_id': ObjectId('64e434c9b6c5bdee4f41aa4e'),
  'date': '2022-01-10',
  'cloud_cover': 17,
  'precipitation': 0.24,
  'min_temp': 9.5,
  'max_temp': 20.84,
  'morning_temp': 13.69,
 

In [30]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_ridedata', 'weather_daily', 'divvy_ridedata_merged']


In [76]:
# Use aggregation pipeline to find top ten start stations 
pipeline = [
    {
        "$group": {
            "_id": "$start_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$start_lat"},
            "longitude": {"$first": "$start_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 11
    }
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

# Print the result
for doc in result:
    print(doc)

{'_id': '', 'count': 833064, 'latitude': 41.93, 'longitude': -87.76}
{'_id': 'Streeter Dr & Grand Ave', 'count': 75237, 'latitude': 41.892278, 'longitude': -87.612043}
{'_id': 'DuSable Lake Shore Dr & Monroe St', 'count': 41279, 'latitude': 41.880958, 'longitude': -87.616743}
{'_id': 'DuSable Lake Shore Dr & North Blvd', 'count': 40090, 'latitude': 41.9117565, 'longitude': -87.62676316666666}
{'_id': 'Michigan Ave & Oak St', 'count': 39661, 'latitude': 41.9008708, 'longitude': -87.623685956}
{'_id': 'Wells St & Concord Ln', 'count': 37515, 'latitude': 41.912133, 'longitude': -87.634656}
{'_id': 'Clark St & Elm St', 'count': 35037, 'latitude': 41.902973, 'longitude': -87.63128}
{'_id': 'Millennium Park', 'count': 35005, 'latitude': 41.8810317, 'longitude': -87.62408432}
{'_id': 'Kingsbury St & Kinzie St', 'count': 33725, 'latitude': 41.88917683258, 'longitude': -87.6385057718}
{'_id': 'Theater on the Lake', 'count': 32976, 'latitude': 41.926277, 'longitude': -87.630834}
{'_id': 'Wells S

In [77]:
# Use aggregation pipeline to find top ten start stations 
pipeline = [
    {
        "$group": {
            "_id": "$end_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 11
    }
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

# Print the result
for doc in result:
    print(doc)

{'_id': '', 'count': 892742, 'latitude': 41.9, 'longitude': -87.64}
{'_id': 'Streeter Dr & Grand Ave', 'count': 75382, 'latitude': 41.892278, 'longitude': -87.612043}
{'_id': 'DuSable Lake Shore Dr & North Blvd', 'count': 42141, 'latitude': 41.911722, 'longitude': -87.626804}
{'_id': 'Michigan Ave & Oak St', 'count': 40127, 'latitude': 41.90096039, 'longitude': -87.62377664}
{'_id': 'DuSable Lake Shore Dr & Monroe St', 'count': 40125, 'latitude': 41.880958, 'longitude': -87.616743}
{'_id': 'Wells St & Concord Ln', 'count': 37421, 'latitude': 41.912133, 'longitude': -87.634656}
{'_id': 'Millennium Park', 'count': 35234, 'latitude': 41.8810317, 'longitude': -87.62408432}
{'_id': 'Clark St & Elm St', 'count': 34490, 'latitude': 41.902973, 'longitude': -87.63128}
{'_id': 'Theater on the Lake', 'count': 32988, 'latitude': 41.926277, 'longitude': -87.630834}
{'_id': 'Kingsbury St & Kinzie St', 'count': 32380, 'latitude': 41.88917683258, 'longitude': -87.6385057718}
{'_id': 'Wells St & Elm St